## Basic practice with Numba and CuPy

In [17]:
import cupy as cp
import numpy as np

In [2]:
from numba import vectorize
import math

SQRT_2PI = np.float32((2*math.pi)**0.5)

@vectorize(['float32(float32, float32, float32)'], target='cuda')
def gaussian_pdf(x, mean, sigma):
    
    return math.exp(-0.5 * ((x - mean) / sigma)**2) / (sigma * SQRT_2PI)

In [8]:
x = np.random.uniform(-3, 3, size=1000000).astype(np.float32)
mean = np.float32(0.0)
sigma = np.float32(1.0)



[ 0.13803269 -1.3787131   1.7333305  -1.4292539  -2.408294    0.234849
  0.66618776  0.69363    -0.34130946  2.0983777 ]


In [5]:
gaussian_pdf(x[0], 0.0, 1.0)

array([0.00943394], dtype=float32)

##### Performance measure with numba and in-built scipy gaussian method

In [6]:
import scipy.stats 
norm_pdf = scipy.stats.norm

%timeit norm_pdf.pdf(x, loc=mean, scale=sigma)

79.2 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit gaussian_pdf(x, mean, sigma)

4.76 ms ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


###### Modularizing function to be vectorize with cuda env

In [9]:
from numba import cuda

#Python  returning tuple is fine with jit device
@cuda.jit(device=True)
def polar_to_cartesian(rho, theta):
    x = rho * math.cos(theta)
    y = rho * math.sin(theta)
    return x, y  

@vectorize(['float32(float32, float32, float32, float32)'], target='cuda')
def polar_distance(rho1, theta1, rho2, theta2):
    x1, y1 = polar_to_cartesian(rho1, theta1)
    x2, y2 = polar_to_cartesian(rho2, theta2)
    
    return ((x1 - x2)**2 + (y1 - y2)**2)**0.5

In [10]:
n = 1000000
rho1 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta1 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)
rho2 = np.random.uniform(0.5, 1.5, size=n).astype(np.float32)
theta2 = np.random.uniform(-np.pi, np.pi, size=n).astype(np.float32)


In [11]:
polar_distance(rho1, theta1, rho2, theta2)

array([1.3069112 , 0.32534564, 1.8428711 , ..., 0.7094915 , 1.7918787 ,
       0.5845356 ], dtype=float32)

In [12]:
%timeit polar_distance(rho1, theta1, rho2, theta2)

6.19 ms ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
gpu_arr=cp.arange(10)

In [14]:
gpu_arr.device

<CUDA Device 0>

In [15]:
cpu_arr=np.arange(10)
gpu_ar=cp.asarray(cpu_arr)

In [16]:
ary_cpu_returned = cp.asnumpy(gpu_ar)

In [19]:
from numba import vectorize

@vectorize(['float32(float32, float32)'], target='cuda')
def add_func(x,y):
    return x+y

In [20]:
n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x


In [21]:
%timeit add_func(x,y)

1.44 ms ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [28]:
#Using numba cuda to compute numpy array on gpu
from numba import cuda

x_cuda=cuda.to_device(x)
y_cuda=cuda.to_device(y)


In [24]:
%timeit add_func(x_cuda,y_cuda)

610 µs ± 814 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
out_device = cuda.device_array(shape=(n,), dtype=np.float32)  

In [26]:
%timeit add_func(x_cuda,y_cuda,out=out_device)

312 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [27]:
out_host = out_device.copy_to_host()
print(out_host[:10])

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]


In [45]:
n = 100000
x_cupy= cp.arange(n)
y_cupy = 2 * x_cupy


def add(x,y):
    return x+y

In [49]:
%timeit cp.add(x_cupy,y_cupy)

21 µs ± 667 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
